## Prática: Criação da configuração experimental e avaliação dos resultados

## Grupo

* Arthur Sabioni
* Eduardo Martins
* Felipe Morais
* Iagor de Sousa
* Marcus Durães

In [1]:
!python3 -m pip install optuna hiplot numpy

Defaulting to user installation because normal site-packages is not writeable


## Parte 1 - Métricas de resultado, método de aprendizado e divisão por folds

**Atividade 1**: Primeiramente, analise a classe `Resultado` que possui os seguintes atributos/propriedades: 
    
   - **mat_confusão**: Retorna a matriz de confusão correpondente. Analise o código e entenda o que representa a linha e a coluna dessa matriz.
    
   - **precisao**:A partir da matriz de confusão, calcula a precisão por classe. Cada indice é o rótulo da classe. Caso o número de elementos previstos com uma determinada classe qualquer `c` seja zero, então `precisao[c] = 0`. Nesses casos, é [lançado um warning](https://docs.python.org/3.7/library/warnings.html) da classe `UndefinedMetricWarning` com uma mensagem que não havia instancias previstas para essa classe.
   - **revocacao**: De forma similar à `precisao`, calcula a revocação por meio da matriz de confusão. Caso o número de elementos dessa classe seja igual a zero, então a revocação para esta class é zero e também deverá ser retornado um warning `UndefinedMetricWarning` com essa informação. 
   - **f1_por_classe**: Retorna, para cada classe, o seu valor F1. Caso a soma da precisão e revocação dessa classe seja zero, deverá ser retornado zero.

Você deverá implementar as seguintes propriedades: 

   - **macro_f1**: Calcula a média do f1 por classe. O método [`np.average`](https://numpy.org/doc/stable/reference/generated/numpy.average.html) pode ajudar.
   - **acuracia**: Calcula a acurácia  por meio da matriz de confusão.



Logo após, execute os seguintes testes unitários: 

para validar o macro F1:

In [2]:
!python3 -m tests TestResultado.test_macro_f1

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Para validar a acurácia: 

In [3]:
!python3 -m tests TestResultado.test_acuracia

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


**Atividade 2 - Classe ScikitLearnAprendizadoDeMaquina: ** O arquivo `metodo.py` é o arquivo que possui os métodos de aprendizado de máquina. A classe `MetodoAprendizadoMaquina` é a classe abstrata para armazenar um método de aprendizado de máquina. Cada instancia, representa um método com seus determinados parametros. A classe `ScikitLearnaprendizadoDemaquina` é responsável por implementar métodos de aprendizado de máquina da API do [Scikit Learn](http://scikit-learn.org). Cada instancia desta classe armazena o respectivo método no atributo `ml_method`.  Por exemplo: 



Assim, você deve implementar o método `eval` da classe `ScikitLearnAprendizadoDeMaquina` para treinar e avaliar. Para isso, você deverá separar a coluna que se refere a classe e as colunas que se referem aos atributos. Logo após você deverá criar o modelo e executar obter a predições. No final, este método retorna uma instancia da classe `Resultado`. 

Execute o código abaixo para verificar o funcionamento deste método: 

In [4]:
!python3 -m tests MetodoTest.test_eval

Macro f1: 0.5982142857142857 Acuracia: 0.6
.
----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


**Atividade 3 - Criação dos folds: ** O arquivo `resultado.py` possui a classe `Fold` que é responsável por armazenar o treino, teste e validação (quando existente). Essa classe possui os seguintes atributos: 

- `df_treino`: Dataframe as instancias de treino. Cada instancia é uma linha e, suas colunas, são seus atributos e a sua classe
- `df_data_to_predict`: Dataframe com as instancias de teste, representada da mesma forma que `df_treino`
- `col_classe`: Coluna que representa a classe alvo nos DataFrames `df_treino` e `df_data_to_predict`
- `arr_folds_validacao`: vetor com os folds de validação. Os folds de validação são também instancias da classe Fold. Tais instancias são construidas a partir do treino - você irá fazer isso nesta atividade. 

** Atividade 3(a): ** Primeiramente, implemente o [método estático](https://daniel-hasan.github.io/cefet-web-grad/classes/python2/#heranca) `gerar_k_folds`. A principio, ignore os parametros  `num_folds_validacao` e `num_repeticoes_validacao`. Este método divide em vários fold os dados `df_dados`. Cada fold deverá ser representado por uma instancia da classe Fold. Deve-se dividir o dataset em $k$ folds (parâmetro  `val_k`) e podem ser feitas $n$ repetições (parâmetro `num_repeticoes`). A escolha das instancias é feita sempre aleatória e, em cada repetição, todos os valores devem estar presentes  em apenas um teste. O treino ficaria com o restante dos valores. Veja abaixo um exemplo se dividirmos em três folds com duas repetições. Foi feito uma função para dividir em três folds e um exemplo em que foi gerado 2 repetições dele.  Para isso, usou-se a função [sample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) para embaralhar os dados e um seed fixo para que sempre embaralhar da mesma forma. Esse seed é essencial para que se garanta a repodutibilidade dos experimentos. 

Diferentemente da função implementada acima, o método a ser implementado por vocês deverá ter uma quantidade qualquer de repetições (parâmetro `num_repeticoes`) e folds por repetição (parâmetro `val_k`). Ao criar o fold, também é necessário saber a coluna da classe, para isso, o parametro `col_classe` armazena seu valor. Implemente o método `gerar_k_folds`. Logo após, execute o seguinte código unitário: 

In [5]:
!python3 -m tests TestFold.test_gerar_k_folds

.
----------------------------------------------------------------------
Ran 1 test in 0.019s

OK


**Atividade 3(b):** Agora, você deverá inicialiar o atributo `arr_folds_validacao` com um vetor de folds de validação, por meio dos dados de treino, de acordo com o número de repetições e folds passados como parametro no construtor. Para isso, invoque o método `gerar_k_folds` no construtor - note que estes folds a serem criados não possuirão validação - possuirão apenas treino e dados a serem previstos (teste). 

Logo após, faça uma pequena modificação no `gerar_k_folds`: os parametros  `num_folds_validacao` e `num_repeticoes_validacao` indicam se o fold a ser criado possuirá validação. Ao instanciar o fold, esses parametros devem ser passados para o contrutor do Fold. Teste a execução a seguir: 

In [6]:
!python3 -m tests TestFold.test_arr_validacao

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK
